In [5]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### First Attempt just using Beautiful Soup

In [6]:
levels = ['college', 'high-school']
sports = ['football', 'basketball', 'baseball', 'womens-basketball', 'volleyball', 'gymnastics',
         'mens-lacrosse', 'womens-lacrosse', 'mens-soccer', 'womens-soccer', 'softball', 'womens-track',
         'mens-golf', 'womens-golf', 'mens-hockey', 'womens-hockey', 'mens-swimming', 'womens-swimming']

all_names = []

for level in levels:
    for sport in sports:

        URL = f'https://www.on3.com/nil/rankings/player/{level}/{sport}/'

        page = requests.get(URL)
        soup = BeautifulSoup(page.content, 'html.parser')
        results = soup.find(class_="NilRankingsPageComponent_nilRankingsList__t14Ao")

        try:
            names_sport = results.find_all('a', class_="MuiTypography-root MuiLink-root MuiLink-underlineNone MuiTypography-h5 MuiTypography-colorPrimary")
            for name in names_sport:
                all_names.append(name)
        except:
            # print("Whoops! No athletes appear to have NIL deal for {} {}".format(level, sport))
            continue
    #end
#end

In [7]:
print(len(all_names))

414


It looks like there were only 414 athletes using this section of the On3 database... This is such a small number of athletes that we should investiage looking at different sections of the website

### Using Selenium & Beatuiful Soup

In [10]:
sports = ['basketball']
#years = ['2022','2023','2024','2025','2026']
years = ['2023']


urls = {}
count = 0
for year in years:
    for sport in sports:
        driver = webdriver.Chrome()
        URL = f'https://www.on3.com/db/rankings/industry-player/{sport}/{year}/'
        driver.get(URL)
        while True:
            try:
                load_more_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, "//span[@class='MuiButton-label' and contains(text(), 'Load More')]"))
                )
                load_more_button.click()
                time.sleep(10)
            except:
                break
            
        page_source = driver.page_source
        page_soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = page_soup.find(class_="PlayerRankings_playerRankings__jvfFg")
        try:
            names_sport = results.find_all('a', class_="MuiTypography-root MuiLink-root MuiLink-underlineHover MuiTypography-h5 MuiTypography-colorPrimary")
            for name in names_sport:
                urls[name.text] = name['href']
                count+=1
        except:
            continue

driver.quit()

In [11]:
print(len(urls))
print(urls)
print(count)

324
{'Isaiah Collier': '/db/isaiah-collier-49422/', 'Justin Edwards': '/db/justin-edwards-69277/', 'Ron Holland': '/db/ron-holland-102172/', 'Aaron Bradshaw': '/db/aaron-bradshaw-350/', 'Matas Buzelis': '/db/matas-buzelis-84972/', 'Cody Williams': '/db/cody-williams-147262/', 'DJ Wagner Jr.': '/db/dj-wagner-jr-28001/', 'Stephon Castle': '/db/stephon-castle-108546/', "Ja'Kobe Walter": '/db/jakobe-walter-50626/', 'Omaha Biliew': '/db/omaha-biliew-94360/', 'Jared McCain': '/db/jared-mccain-56699/', 'Mackenzie Mgbako': '/db/mackenzie-mgbako-81211/', 'Elliot Cadeau': '/db/elliot-cadeau-150874/', 'Xavier Booker': '/db/xavier-booker-147256/', 'Robert Dillingham': '/db/robert-dillingham-101009/', 'Kwame Evans Jr.': '/db/kwame-evans-jr-76260/', 'Sean Stewart': '/db/sean-stewart-105663/', 'Aden Holloway': '/db/aden-holloway-1508/', 'TJ Power': '/db/tj-power-113134/', 'Caleb Foster': '/db/caleb-foster-16985/', 'Jarin Stevenson': '/db/jarin-stevenson-150824/', 'Elmarko Jackson': '/db/elmarko-jacks

In [12]:
# sports = ['basketball']
# #years = ['2022','2023','2024','2025','2026']
# years = ['2023']


# urls = {}

# for year in years:
#     for sport in sports:

        
            
        # page = requests.get(URL)
        # soup = BeautifulSoup(page.content, 'html.parser')
        #results = soup.find(class_="PlayerRankings_playerRankings__jvfFg")
        
        # page_soup = soup(driver.page_source, 'html.parser')
        # print(page_soup)
        # results = page_soup.find(class_="PlayerRankings_playerRankings__jvfFg")
        # try:
        #     names_sport = results.find_all('a', class_="MuiTypography-root MuiLink-root MuiLink-underlineHover MuiTypography-h5 MuiTypography-colorPrimary")
        #     for name in names_sport:
        #         urls[name.text] = name['href']
        # except:
        #     print("Whoops! No athletes appear to have NIL deal for {} {}".format(level, sport))
        #     continue
    #end
#end

In [13]:
# all_info = {}

# for name, url in urls.items():
#     all_info[name] = []
#     URL = f'https://www.on3.com{url}/recruiting'
#     page = requests.get(URL)
#     soup = BeautifulSoup(page.content, 'html.parser')
    
    
    
#     results = soup.find(class_="PlayerRecruiting_playergrid__wYlb_")
#     player_info = results.find_all('span', class_="MuiTypography-root MeasurementInfo_text__dCryI MuiTypography-body1 MuiTypography-colorTextPrimary")
#     for info in player_info:
#         all_info[name].append(info.text)
    
#     results = soup.find(class_="Rankings_container__U2afk")
#     rankings_info = results.find_all('p', class_="MuiTypography-root Rankings_industryRating__9uavm MuiTypography-body1 MuiTypography-colorTextPrimary")
#     for info in rankings_info:
#         all_info[name].append(info.text)
    
    
#     results = soup.find(class_="PlayerInterestsModule_targets__rxz4U")
#     recruitment_info = results.find_all(['a','h6'], class_=["MuiTypography-root MuiLink-root MuiLink-underlineNone PlayerInterestsItem_teamName__FeBHv MuiTypography-h5 MuiTypography-colorPrimary","MuiTypography-root PlayerInterestsItem_predictionLabel__mT201 MuiTypography-subtitle1 MuiTypography-colorTextPrimary"])
#     for info in  recruitment_info:
#         all_info[name].append(info.text)
